In [16]:
import openai
import json

client = openai.OpenAI()
messages = []

In [17]:
def get_weather(city):
    # Dummy implementation for example purposes
    return f"The current weather in {city} is sunny with a temperature of 25°C."

FUNCTION_MAP = {
    "get_weather": get_weather
}

In [18]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather in a given city",
            "parameters": {
                "type": "object", 
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city to get the weather for"
                    }
                },
                "required": ["city"]
            }
        }
    }
]

In [25]:
from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls:
        messages.append({
            "role": "assistant",
            "content": message.content or "",
            "tool_calls": [{
                "id": tool_call.id,
                "type": "function", 
                "function": {
                    "name": tool_call.function.name,
                    "arguments": tool_call.function.arguments
                }
            } for tool_call in message.tool_calls]
        })

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function: {function_name} with arguments: {arguments}")

            try: 
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            print(f"Ran function: {function_name} with result: {result}")

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": result
            })

        call_ai()

    else:
        messages.append({
            "role": "assistant",
            "content": message.content or ""
        })

        if message.content:
            print("AI: " + message.content)
        else:
            print("AI: (no content)")




def call_ai(): 
    response = client.chat.completions.create(
            model="gpt-5-nano",
            messages=messages,
            tools=TOOLS
    )
    process_ai_response(response.choices[0].message)

In [26]:
while True:
    message = input("You: ")
    if message == "exit":
        break
    else:
        messages.append({
            "role": "user",
            "content": message
        })
        print("User: " + message)
        call_ai()

User: 안녕 난 유종석이야
AI: 안녕하세요, 유종석님! 반갑습니다. 어떤 도움을 드릴까요? 날씨 확인, 일정 관리, 텍스트 다듬기, 정보 검색 등 필요하신 게 있으면 말씀해 주세요.
User: 서울 날씨 알려줘
Calling function: get_weather with arguments: {"city":"Seoul"}
Ran function: get_weather with result: The current weather in Seoul is sunny with a temperature of 25°C.
AI: 서울의 현재 날씨는 맑고 기온은 25°C입니다.

다른 도시의 날씨나 시간대별 예보가 필요하시면 알려주세요. 날씨 알림 설정도 도와드릴게요.
